<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/eras/eras/tune/programs/new_dataCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ツール

In [ ]:
def beep():
  from google.colab import output
  output.eval_js('new Audio(\
"https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg")\
.play()') 

In [ ]:
import matplotlib.pyplot as plt
import cv2
def display_ndarray(image):
  img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  plt.imshow(img)
  plt.show()

#ディレクトリ環境設定

In [ ]:
import os
from os import path
from google.colab import drive

def make_dirStruct():
  # gdrive 接続
  if not path.exists('/content/drive'):
    drive.mount('/content/drive')
  else:
    print("Already confirm")
  # colab テンポラリディレクトリの作成
  desk = '/content/desk'
  if not os.path.exists(desk):
    os.mkdir(desk)
  os.chdir(desk)
  print("Created at /content/desk")
  return desk

In [ ]:
desk = make_dirStruct()

Mounted at /content/drive


In [ ]:
class video_format:
  name = "video_format"
  # サンプリングされたCMデータの仕様
  playtime = "15秒"
  displaysize = "(any, any, RGB)"
  videoformat = "any"
  # モデルが扱うCMデータ(上のようなデータは、下のように変換される)
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLOR = "RGB"
  FPS = "2 (FRAME_SIZE / playtime)" # 定義ではなく上から導かれた値

In [ ]:
format1 = video_format()

## [A-1-1]データ取得

In [ ]:
dataPath = '/content/drive/MyDrive/new_categorize'
if not os.path.isdir(dataPath):
  print("no such data path")

get paths of each category

In [ ]:
CATEGORIES = ['product', 'service', 'serviceORproduct', 'unjudgeable']

categoryPaths = []
for each in CATEGORIES:
  thePath = os.path.join(dataPath, each)
  if os.path.isdir(thePath):
    categoryPaths.append(thePath)
  else:
    print("error")
    break
  
if len(CATEGORIES) != len(categoryPaths):
  print("error: in directory struct")
  exit()

#データの可視化

get all 

In [ ]:
data_inEachCategory = []
for eachPath in categoryPaths:
  files = os.listdir(eachPath)
  if '.DS_Store' in files:
    files.remove('.DS_Store')
    print("remove DS_Store")
  else:
    print("no DS_Store")
    
  data_inEachCategory.append(files)

remove DS_Store
['meiji_ポリポリ元気.mp4', 'DAIHATSU_Tanto_Custom.mpg', 'ケンタッキー_食品_カーネリングポテト.mp4', 'pokka sapporo_食品_こんがりパン コーンポタージュ.mp4', 'Honda_N-Box.mpg', '三幸製菓_チーズアーモンド.mp4', 'Kose_化粧品_ESPRIQUE.mp4', 'Honda_N-Custom.mpg', 'pepsi_STRONG_5.0GV.mp4', 'Kao_Primavista_下地.mpg', 'HONDA_車_NBOX.mp4', 'いろはす_飲料_いろはすサイダー.mp4', 'Kao_リーゼ.mpg', '小林製薬_ハレナース.mp4', '大塚製薬_洗髪料_ULOS.mp4', 'ペヤング_食品_ペヤングヌードル.mp4', 'poddi_10分ごはん.mp4', 'gsk_薬品_ボルタレン.mp4', 'NISSAN_CUBE.mpg', 'UHA味覚糖_菓子_e-maのど飴.mp4', 'はごろもフーズ_シーチキンSmile.mp4', 'SUBARU_FORESTER.mp4', 'Reveur_洗髪料_Fraicheur.mp4', '白鶴_酒_まる.mp4', 'セブン&アイHLDGS_食品_ヨークベニマルの鰻.mp4', '白鶴_酒_大吟醸.mp4', 'フジパン_本仕込.mp4', 'ヤクルト_ジョア.mpg', 'キリン_コーヒー_FIRE.mp4', 'タケダ_薬_アリナミンEX PLUS.mp4', '丸亀製麺_釜揚げうどん.mp4', '小林製薬_サラサーティ.mp4', 'Kao_Segreta.mp4', '雪印メグミルク_食品_さけるチーズ.mp4', '資生堂_Perfect_Watery_Oil.mpg', 'UHA味覚糖_ぷっちょ とろけるグミ.mp4', 'Eisai_薬品_チョコラBBプラス.mp4', '森永乳業_飲料_アロエステ.mp4', '雪印メグミルク_食品_ガセリ菌SPヨーグルト.mp4', '銀のスプーン_三ツ星グルメ.mp4', 'ニトリ_わんぱく組.mp4', '第一三共_第一三共

In [ ]:
nums_data = []
nums = 0
for numdata in data_inEachCategory:
  nums_data.append(len(numdata))
  nums = nums + len(numdata)

In [ ]:
import matplotlib.pyplot as plt
 
left = [1, 2, 3, 4]
height = nums_data

labels = ['product', 'service', 'serviceORproduct', 'unjudgeable']
 
plt.bar(left, height, width=0.5, color='#0096c8',
        edgecolor='b', linewidth=2, tick_label=labels)

plt.text(3, 60, 
         "\n product: 82\n service: 16\n serviceORproduct: 10\n unjudgeable: 4\n all: 112",
         size=10)

plt.show()

print("Number product: ", nums_data[0])
print("Number service: ", nums_data[1])
print("Number serviceORproduct: ", nums_data[2])
print("Number unjudgeable: ", nums_data[3])
print("-----------------------------------------------------")
print("Number All: ", nums)

plt.savefig("img.png")

#内部の変数として、適切な形にして取り込む

In [ ]:
import math
import cv2
def makeupVideo(video):
  framesize = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  cutTiming = math.floor(framesize / format1.FRAME_SIZE)
  cutTail = cutTiming * format1.FRAME_SIZE

  normalizedFrames = []
  count = 0

  # フレーム３０枚分を１つの動画から間伐する
  while count <= cutTail - 1:
    IO, frame = video.read()
    if (count % cutTiming) == 0: 
      if not IO:
        print("error") 
        break
      else: 
        fixedFrame = cv2.resize(frame, dsize=(format1.WIDTH, format1.HEIGHT))
        display_ndarray(fixedFrame)
        fixedFrame = fixedFrame.flatten().astype(np.float32)/255.0
        normalizedFrames.append(fixedFrame)
    else:
      pass

    count += 1
  return np.array(normalizedFrames)

In [ ]:
%%time
# Wall time: 9min 28s
# Wall time: 8min 12s
videoContener = []
labelContener = []
for index, eachPath in enumerate(categoryPaths):
  files = data_inEachCategory[index]
  print(eachPath, "& label =", index)
  print("=============================================")
  for f in files:
    theFile = os.path.join(eachPath, f)
    print("searching ", theFile)
    if os.path.isfile(theFile):
      print('@exists.')
      print("@loading from physical.")
      innnerData_video = cv2.VideoCapture(theFile)
      label = index
      innnerData_frames = makeupVideo(innnerData_video)

      videoContener.append(innnerData_frames)
      labelContener.append(label)

      print("finish")
      print("-------------------------------------------\n")
    # break
  # /for f
  # break
# /for index
beep()

チェック

In [ ]:
len(videoContener)
len(labelContener)

112

# パッキング

In [ ]:
def save_data(npdata, saveName):
  np.savez(os.path.join(desk, saveName), *npdata)


In [ ]:
save_data(np.array(videoContener), "normalTrainVideos")
save_data(np.array(labelContener), "normalTrainLabels")

#ストレージに保存

In [ ]:
import shutil

shelf = '/content/drive/MyDrive/colab'
book = 'data0110'
shelf_book = os.path.join(shelf, book)
if not os.path.exists(shelf_book):
  os.mkdir(shelf_book)



In [ ]:
wants_paths = [
"/content/desk/normalTrainLabels.npz",
"/content/desk/normalTrainVideos.npz",
"/content/desk/img.png"
]

for want in wants_paths:
  shutil.copy2(
      want,
      shelf_book
  )